# TIEGCM analysis and visualization

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from netCDF4 import Dataset

In [3]:
from plotly.offline import plot, iplot
import plotly.graph_objs as go

from plotly.graph_objs import Layout

In [4]:
import plotly

In [5]:
import scipy
import numpy as np

from scipy.interpolate import griddata, LinearNDInterpolator

In [6]:
import pandas as pd

In [7]:
from util import *

In [9]:
plotly.offline.init_notebook_mode(connected=True)

In [10]:
directory = "sample_data/jasoon_shim_052317_IT_10/out/"

In [12]:
ls sample_data/jasoon_shim_052317_IT_10/out/

p001.nc  s001.nc


In [13]:
pfile = directory + 'p001.nc'
sfile = directory + 's001.nc'
print pfile
print sfile

sample_data/jasoon_shim_052317_IT_10/out/p001.nc
sample_data/jasoon_shim_052317_IT_10/out/s001.nc


In [14]:
rootgrp = Dataset(sfile, 'r')

In [15]:
dimensional = group_dimensional(rootgrp)
dimensional.keys()

['0-d', '3-d', '4-d', '1-d', '2-d']

In [16]:
describe(rootgrp, dimensional['0-d'])

,units,long_name,shape,min,max
timestep,seconds,timestep,(),6.000000e+01,6.000000e+01
p0,millibars,Reference pressure,(),5.000000e-07,5.000000e-07
p0_model,microbars,Reference pressure (as used by the model),(),5.000000e-04,5.000000e-04
grav,cm/s,gravitational acceleration,(),8.700000e+02,8.700000e+02
LBC,None,"Interface level of t,u,v lower boundary condition",(),-7.000000e+00,-7.000000e+00


In [17]:
np.array(rootgrp.variables['ut'])

array([ 0.33333333,  0.66666667,  1.        ,  1.33333333,  1.66666667,
        2.        ,  2.33333333,  2.66666667,  3.        ,  3.33333333,
        3.66666667,  4.        ,  4.33333333,  4.66666667,  5.        ,
        5.33333333,  5.66666667,  6.        ,  6.33333333,  6.66666667,
        7.        ,  7.33333333,  7.66666667,  8.        ])

In [18]:
np.array(rootgrp.variables['day'])

array([275, 275, 275, 275, 275, 275, 275, 275, 275, 275, 275, 275, 275,
       275, 275, 275, 275, 275, 275, 275, 275, 275, 275, 275], dtype=int32)

In [19]:
describe(rootgrp, dimensional['1-d'])

,units,long_name,shape,min,max
time,minutes since 2012-10-1 0:0:0,time,"(24,)",2.000000e+01,4.800000e+02
lon,degrees_east,"geographic longitude (-west, +east)","(72,)",-1.800000e+02,1.750000e+02
lat,degrees_north,"geographic latitude (-south, +north)","(36,)",-8.750000e+01,8.750000e+01
lev,,midpoint levels,"(29,)",-6.750000e+00,7.250000e+00
ilev,,interface levels,"(29,)",-7.000000e+00,7.000000e+00
mlon,degrees_east,"magnetic longitude (-west, +east)","(81,)",-1.800000e+02,1.800000e+02
mlat,degrees_north,"magnetic latitude (-south, +north)","(97,)",-9.000000e+01,9.000000e+01
mlev,,magnetic midpoint levels,"(32,)",-8.250000e+00,7.250000e+00
imlev,,magnetic interface levels,"(32,)",-8.500000e+00,7.000000e+00
year,None,calendar year,"(24,)",2.012000e+03,2.012000e+03


Each file represents 8 hours  of simulation time. There are 24 time steps in each file, corresponding to one step every 20 minutes. 

In [20]:
rootgrp.variables['time']

<type 'netCDF4._netCDF4.Variable'>
float64 time(time)
    long_name: time
    units: minutes since 2012-10-1 0:0:0
    initial_year: 2012
    initial_day: 275
    initial_mtime: [275   0   0]
unlimited dimensions: time
current shape = (24,)
filling off

In [21]:
describe(rootgrp, 
         ['time', 'ilev', 'lev','lat', 'lon'], 
         ['units', 'short_name', 'long_name', 'shape', 'formula'])

,units,short_name,long_name,shape,formula,min,max
time,minutes since 2012-10-1 0:0:0,None,time,"(24,)",None,20.00,480.00
ilev,,ln(p0/p),interface levels,"(29,)",p(k) = p0 * exp(-ilev(k)),-7.00,7.00
lev,,ln(p0/p),midpoint levels,"(29,)",p(k) = p0 * exp(-lev(k)),-6.75,7.25
lat,degrees_north,None,"geographic latitude (-south, +north)","(36,)",None,-87.50,87.50
lon,degrees_east,None,"geographic longitude (-west, +east)","(72,)",None,-180.00,175.00


### 2D variables

In [55]:
def to_date(year, doy):
    return pd.to_datetime(str(year) + str(doy),format ='%Y%j')
to_date(start_year, start_day)

Timestamp('2012-10-01 00:00:00')

In [53]:
pd.Timedelta(hours = 3.5)

Timedelta('0 days 03:30:00')

In [44]:
rootgrp.variables['year'][[0, -1]]

array([2012, 2012], dtype=int32)

In [60]:
def get_time_range(self):
    start_year = self.rootgrp.variables['year'][0]
    start_day = self.rootgrp.variables['day'][0]
    end_ut = self.rootgrp.variables['ut'][-1]
    start = to_date(start_year, start_day)
    end = start + pd.Timedelta(hours = end_ut)
    return start, end

get_time_range(tiegcm)

(Timestamp('2012-10-01 00:00:00'), Timestamp('2012-10-01 08:00:00'))

In [22]:
describe(rootgrp, dimensional['2-d'])

,units,long_name,shape,min,max
mtime,"day, hour, minute","model times (day, hour, minute)","(24, 3)",0.000000e+00,2.750000e+02
write_date,None,Date and time each history was written,"(24, 24)",NaN,NaN
dtide,None,"amplitude and phase of diurnal tide mode (1,1)","(24, 2)",0.000000e+00,0.000000e+00
sdtide,None,amplitudes and phases of semi-diurnal tide,"(24, 10)",0.000000e+00,0.000000e+00
ncep_ncfile,None,ncep data file,"(24, 1024)",NaN,NaN
gpi_ncfile,None,GeoPhysical Indices data file,"(24, 1024)",NaN,NaN
saber_ncfile,None,SABER lbc data file,"(24, 1024)",NaN,NaN
tidi_ncfile,None,TIDI lbc data file,"(24, 1024)",NaN,NaN
bgrddata_ncfile,None,background lbc data file,"(24, 1024)",NaN,NaN
ctmt_ncfile,None,CTMT lbc data file,"(24, 1024)",NaN,NaN


### 3D shape
3d arrays correspond in shape to [time, lat, lon]

In [23]:
describe(rootgrp, dimensional['3-d'])

,units,long_name,shape,min,max
TEC,1/cm2,TEC: Total Electron Content,"(24, 36, 72)",1.375009e+12,6.294050e+13
QJOULE_INTEG,erg/cm2/s,Height-integrated Joule Heating,"(24, 36, 72)",4.280629e-04,7.633630e+01
EFLUX,erg/cm2/s,Aurora Energy Flux,"(24, 36, 72)",0.000000e+00,1.332358e+01
HMF2,km,HMF2: Height of the F2 Layer,"(24, 36, 72)",1.236165e+02,5.223236e+02
NMF2,1/cm3,NMF2: Peak Density of the F2 Layer,"(24, 36, 72)",4.939089e+04,2.579142e+06
TLBC,K,Lower boundary condition of TN,"(24, 36, 72)",1.564640e+02,2.036720e+02
ULBC,cm/s,Lower boundary condition of UN,"(24, 36, 72)",-5.879154e+03,4.315232e+03
VLBC,cm/s,Lower boundary condition of VN,"(24, 36, 72)",-7.275964e+03,7.556053e+03
TLBC_NM,K,Lower boundary condition of TN (TIME N-1),"(24, 36, 72)",1.564839e+02,2.036565e+02
ULBC_NM,cm/s,Lower boundary condition of UN (TIME N-1),"(24, 36, 72)",-5.873854e+03,4.313503e+03


4-d shape corresponds to [time, ilev, lat, lon]

In [26]:
describe(rootgrp, dimensional['4-d'])

,units,long_name,shape,min,max
TN,K,NEUTRAL TEMPERATURE,"(24, 29, 36, 72)",1.475614e+02,1.385325e+03
UN,cm/s,NEUTRAL ZONAL WIND (+EAST),"(24, 29, 36, 72)",-6.970348e+04,6.350308e+04
VN,cm/s,NEUTRAL MERIDIONAL WIND (+NORTH),"(24, 29, 36, 72)",-7.142820e+04,6.891355e+04
O1,mmr,ATOMIC OXYGEN,"(24, 29, 36, 72)",1.494940e-03,9.463220e-01
NO,mmr,NITRIC OXIDE,"(24, 29, 36, 72)",3.516701e-08,1.583549e-03
N4S,mmr,N4S,"(24, 29, 36, 72)",1.000000e-12,2.071025e-02
HE,mmr,HELIUM,"(24, 29, 36, 72)",1.148865e-06,5.126053e-01
NE,cm-3,ELECTRON DENSITY,"(24, 29, 36, 72)",7.747815e+02,2.578788e+06
TE,K,ELECTRON TEMPERATURE,"(24, 29, 36, 72)",1.475751e+02,4.280271e+03
TI,K,ION TEMPERATURE,"(24, 29, 36, 72)",1.475666e+02,3.397951e+03


In [ ]:
rootgrp.variables['O2N']

In [ ]:
rootgrp.variables['N2N']

In [ ]:
lat = np.array(rootgrp.variables['lat'])
lon = np.array(rootgrp.variables['lon'])
ilev = np.array(rootgrp.variables['ilev'])

In [ ]:
z = np.array(rootgrp.variables['Z'])

In [ ]:
limits = pd.DataFrame(dict(latitude = [lat.min(), lat.max()], 
                           longitude = [lon.min(), lon.max()],
                          z =[ z.min(), z.max()]), index = ['min', 'max'])
limits

In [ ]:
point = (0, 0, (z.max() + z.min())/2)
point

In [ ]:
from collections import namedtuple

In [ ]:
Point4D = namedtuple("Point4D", ['time','height', 'latitude','longitude'])
Point3D = namedtuple("Point3D", ['height', 'latitude','longitude'])
Point2D =  namedtuple("Point2D", ['latitude','longitude'])

point = Point3D( height=41175836.0, latitude=87, longitude=170)
point2D = Point2D(latitude=87, longitude=170)

In [ ]:
point

# Visualize results from tiegcm

## Fixed height interpolation

In [ ]:
lat, lon = np.meshgrid(tiegcm.lat[1:-1], tiegcm.lon)

In [ ]:
z_mid = 400*1e3*1e2 #cm

In [ ]:
points = [Point3D(z_mid, p[0], p[1]) for p in zip(lat.ravel(), lon.ravel())]
points[330]
print len(points)

In [ ]:
points[0]

In [ ]:
variable = 'DEN'

In [ ]:
ne = [tiegcm.time_interpolate(p, variable, 3.5) for p in points]

In [ ]:
min(ne), max(ne)

In [ ]:
phi = lon*np.pi/180
phi.min(), phi.max()

In [ ]:
theta = lat*np.pi/180
theta.min(), theta.max()

In [ ]:
r_E = 637.1e6
r = (r_E + z_mid)/r_E
r

In [ ]:
x = r * np.cos(theta) * np.cos(phi)
y = r * np.cos(theta) * np.sin(phi)
z = r * np.sin(theta)

In [ ]:
lat.shape

In [ ]:
np.array(ne).max()

In [ ]:
trace1 = go.Surface(
    x = x,
    y = y,
    z = z,
    surfacecolor = np.array(ne).reshape(lat.shape),
    colorbar = dict(title = variable + ' [gm/cm3]')
)

In [ ]:
def to_range(x, start, limit):
    result  = start + (x - start) % (limit - start)
    print x, '->', [start, limit], '=', result
    return result
    

r = [0, 2.5]
to_range(3.0, *r)
to_range(-.1, *r)

In [ ]:
r = [-180, 180]
to_range(185, *r)
to_range(-185,*r)

In [ ]:
r = [tiegcm.lon.min(), tiegcm.lon.max()]
to_range(180, *r)
to_range(170, *r)
to_range(176, *r)

We need to get the min longitude slice and append it to the variable

In [ ]:
fig = go.Figure(data=[trace1], layout = go.Layout(title = 'TIEGCM'))

iplot(fig)

The above interpolation is linear using a deluanay triangulation in lat, lon, z. You should probably be doing the interpolation in R-3 by converting from lat, lon, z, to x,y,z. This means the convex hull rule will provide interpolations all the way to the center of the earth.

For time interpolation, you should probably use two time steps at a time?

# Fortran interpolator

look at  htint.F in in  /ccmcshare/MODELS_at_CCMC/TIEGCM/tgcmproc2.3_f90  and tgcmproc2.9_idl.


### htint.F header

```fortran

ccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc
c
c Routines to do linear or log interpolation of gcm fields to 
c a linear height scale. Dimensions of fin, fht, and fout vary
c depending on routine called.
c
c glbhtint() is global height interpolator:
c   fin and fht (nlon,nzp,nlat) = global input and gcm heights fields
c   fout(nlon,nlat,nhts) = global interpolated field returned
c
c glbhtin() is same as glbhtint except that the intput fields fin and fht
c   are dimensioned (nlon,nlat,nzp) rather than (nlon,nzp,nlat). This is
c   more convenient for some codes, e.g. "super_procs" gcm8proc, gcm9proc,
c   etc.
c
c cuthtint() is height interpolator at some slice of the grid (e.g.,
c   along a latitude (idim1 = imx) or longitude (idim1 = jmx)
c   fin and fht (idim1,nzp) = input and gcm heights fields (at some cut)
c   fout(idim1,nhts) = interpolated field returned
c
c intloc() interpolates fields at some location (lat,lon gcm grid point)
c   
c Inputs common to all routines (dimensions vary):
c
c   fin: the gcm field at constant pressures
c   fht: gcm heights at constant pressures
c   hts: the height scale at which to do the interpolation
c   nhts: number of heights in hts
c   logint: if logint > 0, do log (exponential) interpolation, 
c           otherwise linear  
c   spval: if != 0., is special value returned in fout if 
c          ht(i) > max gcm height, or ht(i) < min gcm height. 
c          if = 0., extrapolation will be attempted if ht(i) out of range
c   iprnt: print stuff out if > 0 (e.g., if spval is used)
c
c Outputs common to all routines (dimension of fout varies):
c
c   fout: returned field, interpolated to desired height scale
c   ier: non-zero if an error has occurred
c
c Required routines:
c   fminmax(), bracket()
c
c Ben Foster (ncar/hao)
c foster@ncar.ucar.edu
c 303-497-1595
c
ccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc
c
```
### subroutine glbhtint

```fortran
c
c Do linear or log interpolation of fin to constant height surfaces
c   in hts(nhts). Interpolated field returned in fout.
c
c On input:
c   fin(nlon,nzp,nlat) = global input field
c   fht(nlon,nzp,nlat) = global heights field
c   nlon,nzp,nlat = dimensions of fin and fht
c   hts(nhts) = heights at which to interpolate
c   logint <= 0 -> do linear interpolation,
c                  otherwise do exponential (log) interpolation
c   spval: If spval is non-zero and hts(k) is not within fht range
c          at the current grid point, then fout gets spval at the
c          current grid point.
c          If spval=0 and hts(k) is out of range, then
c          fout will be extrapolated to hts(k)
c   iprnt = 1 -> some error msgs will be printed if necessary
c
c On output:
c   fout(nlon,nlat,nhts) = contains interpolated field
c   ier = non-zero if an error has occurred
c
dimension fin(nlon,nzp,nlat),fht(nlon,nzp,nlat),
+  fout(nlon,nlat,nhts),hts(nhts)
dimension col(nzp)
c
do j=1,nlat
do i=1,nlon
          col(:) = fht(i,:,j)
          call fminmax(col,nzp,htmin,htmax)
          do k=1,nhts
            if (hts(k).lt.htmin.or.hts(k).gt.htmax) then
              if (spval.ne.0.) then
                fout(i,j,k) = spval
                goto 100
              else
                k0 = nzp-1
                k1 = nzp
              endif
            else
              call bracket(hts(k),col,nzp,1,k0,k1,ierr)
              if (ierr.ne.0) then
                if (iprnt.gt.0) write(6,"('glbhtint: error from',
     +            ' bracket: i j=',2i3,' k=',i3,' hts(k)=',f8.2,
     +            ' hts col=',/(6f10.3))") i,j,k,hts(k),col
                fout(i,j,k) = spval
                goto 100
              endif
            endif
c
c Do linear or log interpolation:
c
            if (logint.gt.0) then
              if (fin(i,k1,j).gt.0..and.fin(i,k1,j).ne.spval.and.
     +            fin(i,k0,j).gt.0..and.fin(i,k0,j).ne.spval) then
                exparg = (alog(fin(i,k1,j) / fin(i,k0,j)) /
     +                   (col(k1)-col(k0))) * (hts(k)-col(k0))
                fout(i,j,k) = fin(i,k0,j)*exp(exparg)
              else
                fout(i,j,k) = spval
              endif
            else
              f1 = (hts(k)-col(k0)) / (col(k1)-col(k0))
              fout(i,j,k) = f1*fin(i,k1,j) + (1.-f1)*fin(i,k0,j)
            endif
 100        continue
          enddo
        enddo
      enddo
      return
      end
c
```

### subroutine fminmax(f,n,rmin,rmax)
```fortran
!
! Return min and max of array f(n), excluding any values == spval:
!
      use proc, only: spval
      implicit none
      integer, intent(in)  :: n
      real,    intent(in)  :: f(n)
      real,    intent(out) :: rmin,rmax
      integer i
!
!     rmin = spval
!     rmax = -spval
!
! On dataproc with 4-byte reals, huge(x) = 0.34028235E+39
      rmin = huge(rmin)
      rmax = -rmin
      do i=1,n
        if (f(i).ne.spval.and.f(i).gt.rmax) rmax = f(i)
        if (f(i).ne.spval.and.f(i).lt.rmin) rmin = f(i)
      enddo
      return
      end subroutine fminmax
```

### subroutine bracket(x,xx,nx,inc,n1,n2,ier)
```fortran   
      implicit none
!
! Bracket x in xx(nx), returning lower index in n1 and upper index in n2
! If inc > 0 -> array increases from bottom to top,
! If inc <= 0 -> array increases from top to bottom
!
      integer,intent(in) :: nx,inc
      integer,intent(out) :: n1,n2,ier
      real,intent(in) :: x,xx(nx)
      integer :: i
!
! Array increases from bottom to top:
      ier = 0
      if (inc.gt.0) then
        if (x.lt.xx(1)) then
          n1 = 1
          n2 = 2
          ier = 1
          return
        endif
        if (x.gt.xx(nx)) then
          n1 = nx-1
          n2 = nx
          ier = 2
          return
        endif
        do i=1,nx-1
          if (x.ge.xx(i).and.x.le.xx(i+1)) then
            n1 = i
            n2 = i+1
            return
          endif
        enddo
!
! Array increases from top to bottom:
      else
        if (x.gt.xx(1)) then
          n1 = 1
          n2 = 2
          ier = 3
          return
        endif
        if (x.lt.xx(nx)) then
          n1 = nx-1
          n2 = nx
          ier = 4
          return
        endif
        do i=1,nx-1
          if (x.le.xx(i).and.x.ge.xx(i+1)) then
            n1 = i
            n2 = i+1
            return
          endif
        enddo
      endif
      return
      end subroutine bracket

```

# Binding interpolator functions to C++

http://pybind11.readthedocs.io/en/master/advanced/pycpp/object.html

## Compiling and importing a c++ module into python with cppimport
https://github.com/tbenthompson/cppimport

All the steps can be performed in python using the following
```python
import cppimport
somecode = cppimport.imp("somecode") #This will pause for a moment to compile the module
somecode.square(9)
81
```

## Compiling with cmake
CMakeLists.txt:

```cmake
cmake_minimum_required(VERSION 2.8.12)
project(example)
set(PYBIND11_CPP_STANDARD -std=c++11)
add_subdirectory(pybind11)
pybind11_add_module(example example.cpp)

```
Then compile with these steps
```console
(sunpy):mkdir build
(sunpy):cd build
(sunpy):cmake .. -DPYBIND11_PYTHON_VERSION=2.7 -DCMAKE_CXX_COMPILER=clang -DCMAKE_C_COMPILER=clang -DCMAKE_LIBRARY_OUTPUT_DIRECTORY=..
(sunpy):make
```
Then import from python
```python
import example
example.add(3,5)
```

In [ ]:
import ccmc_interpolator

In [ ]:
from collections import namedtuple

In [ ]:
tiegcm = ccmc_interpolator.TIEGCM_cpp('sample_data/jasoon_shim_052317_IT_10/out/s001.nc')

In [ ]:
tiegcm.name

In [ ]:
z_test = 39005780.

In [ ]:
Point3D = namedtuple("Point3D", ['height', 'latitude','longitude'])
Point4D = namedtuple("Point4D", ['height','latitude','longitude', 'time'])

In [ ]:
point = Point4D(z_test,  87.,  170., 3.5 )

In [ ]:
tiegcm.interpolate('NE', *point)

In [ ]:
result

# Interpolation at the poles

Currently this results in a qhull error. Apparently, there are no cells that reach the poles. We will fill these using Lutz's algorithm.

    ; at-pole boundary conditions
    for iq=0,n_q-1 do begin
    ; pole_bc=0: zero value
        if pole_bc[iq] eq 0 then begin
            fields[iq,*,0,*]=0.
            fields[iq,*,ny_blk-1,*]=0.
        endif
    ; pole_bc=1: average value that Masha mentioned
        if pole_bc[iq] eq 1 then begin
            for iz=0,nz_blk-1 do begin
                fields[iq,*,0,iz]=total(fields[iq,0:nx_blk-2,1,iz])/(nx_blk-1)
                fields[iq,*,ny_blk-1,iz]=total(fields[iq,0:nx-2,ny_blk-2,iz])/(nx_blk-1)
            endfor
        endif
    ; pole_bc=2: no-derivative 
        if pole_bc[iq] eq 2 then begin
                fields[iq,*,0,*]       =fields[iq,*,1,*]
                fields[iq,*,ny_blk-1,*]=fields[iq,*,ny_blk-2,*]
        endif
    endfor
    

Variable polar boundary coditions

    Variable          pole_bc
    T_n       1
    Vn_Lon       2
    Vn_Lat       2
    rho(O)       1
    rho(NO)       1
    rho(N4S)       1
    HE       1
    N_e       1
    T_e       1
    T_i       1
    TEC       1
    rho(O2)       1
    O2P_ELD       1
    Omega       1
    PHI       1
    Vi_Lon       2
    Vi_Lat       2
    Vi_IP       1
    N(O+)       1
    N(N2+)       1
    N(N+)       1
    N(NO+)       1
    SIGMA_PED       1
    SIGMA_HAL       1
    DEN       1
    QJOULE       1
    Z       1
    ZG       1
    O_N2       1
    QJOULE_INTEG       1
    EFLUX       1
    HMF2       1
    NMF2       1
    N2D_ELD       1
    O2N       1
    N2N       1
    T_lbc       1
    Vlbc_Lon       2
    Vlbc_Lat       2
    TN_lbc       1
    VN,lbc_Lon       2
    VN,lbc_Lat       2

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
from tiegcm import TIEGCM, Point3D, Point4D, Slice_key4D, ColumnSlice4D, ColumnSlice3D

In [ ]:
from util import average_longitude

In [ ]:
from tiegcm import TIEGCM

test_file = "sample_data/jasoon_shim_052317_IT_10/out/s001.nc"
tiegcm = TIEGCM(test_file)

In [ ]:
from scipy.spatial import kdtree

from scipy.interpolate import LinearNDInterpolator
time = 3.5

In [ ]:
z_max = tiegcm.z.max()
z_test = 1.1*z_max
p4 = Point4D(time, z_test, 20.5, .5)

column_slicer = tiegcm.get_column_slicer_4D(p4)
z_column, lat_column, lon_column = tiegcm.get_3D_column(column_slicer)

time_index = column_slicer.time.start

In [ ]:
z = tiegcm.z[time_index, -1, :, :]
lat = tiegcm.lat_[-1]
lon = tiegcm.lon_[-1]

outer_points = np.array(zip(z.ravel()/z.max(), lat.ravel(), lon.ravel()))

tree = scipy.spatial.KDTree(outer_points)

z_test = 1.1*tiegcm.z[time_index, -1, :, :].max() # high altitude test

p4 = Point4D(time, z_test, 20.5, .5)
p = Point3D(*p4[1:])

distances, vertices = tree.query(p, p = 1, k = 3)

coord_indices = np.array(zip(*np.unravel_index(vertices, z.shape)))

lat_indices, lon_indices = coord_indices[:,0], coord_indices[:,1]

lnd_lat = LinearNDInterpolator(tree.data[vertices][:,1:], tiegcm.lat_[-1][lat_indices, lon_indices])
lnd_lon = LinearNDInterpolator(tree.data[vertices][:,1:], tiegcm.lon_[-1][lat_indices, lon_indices])
lnd_z = LinearNDInterpolator(tree.data[vertices][:,1:], tiegcm.z[time_index,-1, lat_indices, lon_indices])
assert (float(lnd_lat(p[1:])) == p.latitude)
assert (float(lnd_lon(p[1:])) == p.longitude)

In [ ]:
zz_top = z_column[:,-1,:,:]

In [ ]:
assert zz_top.min() <= lnd_z(p[1:]) <= zz_top.max()

compare with column

In [ ]:
pd.Series(column_slicer._asdict()).to_frame().T

In [ ]:
tiegcm.z[time_index,-1,lat_indices, lon_indices]

In [ ]:
tiegcm.z[time_index,-1,:,:].min(), tiegcm.z[time_index,-1,:,:].max()

In [ ]:
tree.data[vertices][:,1:]

In [ ]:
z_column.shape, lat_column.shape, lon_column.shape

In [ ]:
lat_column[-1], lon_column[-1]

In [ ]:
tiegcm.z[time_index, -1, :, :].max()

In [ ]:
height_index = -1

norm = tiegcm.z[time_index, height_index, :, :].max()

In [ ]:
data = [go.Surface( z=tiegcm.z[time_index, height_index, :, :]/norm,
                    x=tiegcm.lon,
                    y=tiegcm.lat,
                    opacity = .9,
                   name = 'max_height'
                    ),
        go.Surface( z=z_column[0, height_index, :, :]/norm,
                    x=lon_column[height_index],
                    y = lat_column[height_index],
                    opacity = 1,
                   name = 'column_height',
                    ),
        go.Scatter3d(x = [p.longitude], y =[ p.latitude],z = [p.height/norm], name = 'query', mode = 'markers'),
        go.Scatter3d(x = [lon[c[0], c[1]] for c in coord_indices], 
                     y = [lat[c[0], c[1]] for c in coord_indices], 
                     z = tiegcm.z[time_index,height_index, lat_indices, lon_indices]/norm, mode = 'markers')
]

iplot(data)

In [ ]:
p, time

In [ ]:
tiegcm.time_interpolate_high_altitude(p, 'Z', time)

# Scale height

Scale height given by 

$H(h) = KT_{top}/M_xg(h)$

$g(h) = g_0(r_e/(r_e+h))^2$

$g_0 = ?$

In [ ]:
p

In [ ]:
def scale_height(self, p, time, variable):
    print variable
    T = self.time_interpolate_high_altitude(p, 'TN', time)
    return T
scale_height(tiegcm, p, time, 'O2')

In [ ]:
tn = tiegcm.rootgrp.variables['TN']

In [ ]:
tn.__array__().max()

## Interpolator Call Graph


### time_interpolate(point, time)
* retrieves slice_key corresponding to point,time
* scales height down using self.z_scale
* wraps longitude into valid range
* calls `time_interpolators[slice_key](p, time)`

### self.time_interpolators[slice_key]
* a subclass of defaultdict
* if slice_key is not found: calls self.create_3Dtime_interpolator(slice_key)
* stores result

slice_key is a 4D slice that looks like this

In [ ]:
pd.Series(dict(time = (slice, (8,11, None)), 
               height = (slice, (None, None, None)),
               latitude = (slice, (35, 37, None)),
               longitude = (slice, (65, 67, None)),
               variable = 'Z'
              )).to_frame().T

### create_3Dtime_interpolator
* takes a 4d slice key and returns a TimeInterpolator object
* constructs two 3D interpolators corresponding to two time steps: calls self.get_3D_interpolator(slice_key, i) i $\in$ [0,1]
* returns a TimeInterpolator(interpolators)

### get_3D_interpolator(slice_key, i = 0)
* retrieves spatial data corresponding to time slice 
* z_column, lat_column, lon_column, i

### get_delaunay_3D
* scales the z_column by self.z_scale
* martials the vertices into points
* creates an instance of Delaunay(points)

## Cartesian interpolation

We need to convert the positions from spherical to cartesian in two place:
* time_interpolate
* get_delaunay_3D

We may also need the inverse transformation for testing

In [ ]:
from collections import namedtuple

In [ ]:
Point3DCartesian = namedtuple("Point3DCartesian", ['x','y','z'])
Point3DSpherical = namedtuple("Point3DSpherical", ['r','theta','phi'])

In [ ]:
from tiegcm import geo_to_spherical, spherical_to_cartesian, geo_to_cartesian

In [ ]:
geo_to_cartesian(Point3D(0, 90, 0))

for $ r \in [0, \inf), \theta \in [0, \pi], \phi \in [0, 2\pi) $
$$ x = r sin \theta cos \phi $$

$$ y = r sin \theta sin \phi $$

$$ z = r cos \theta $$


In [ ]:
print 'lat in', tiegcm.lat.min(), tiegcm.lat.max()
print 'lon in', tiegcm.lon.min(), tiegcm.lon.max()

In [ ]:
npoints = 10
h = np.linspace(0, 5, npoints)
lat = np.linspace(-25, 90, npoints)
lon = np.linspace(-30, 30, npoints)
geo = Point3D(h,lat,lon)

In [ ]:
pd.concat([pd.DataFrame.from_dict(geo._asdict()),
           pd.DataFrame.from_dict(geo_to_spherical(geo)._asdict()),
           pd.DataFrame.from_dict(geo_to_cartesian(geo)._asdict()),], axis = 1)

# Wrap error

In [11]:
from tiegcm.tiegcm import TIEGCM
import numpy.ma as ma
from util import boundary_conditions

test_file = "sample_data/jasoon_shim_052317_IT_10/out/s001.nc"
tiegcm = TIEGCM(test_file)

In [ ]:
ne = tiegcm.rootgrp.variables['NE'].__array__()

In [ ]:
tn = tiegcm.rootgrp.variables['TN'].__array__()

In [ ]:
if tn[:,-1,:,:].mask.all():
    tn[:,-1,:,:] = tn[:,-2,:,:].data

In [ ]:
tn.filled(np.nan)

In [ ]:
ne.max()

In [ ]:
tn.max()

In [ ]:
tn.compress(~tn.mask.ravel()).shape

In [ ]:
tiegcm.set_variable_boundary_condition('TN',verbose = True)

In [ ]:
test_file = "sample_data/jasoon_shim_052317_IT_10/out/s001.nc"
tiegcm = TIEGCM(test_file)

In [ ]:
z_max = tiegcm.z.max()
z_test = 1.1*z_max

point = Point3D(z_test, -20.5, .5)
point2 = Point3D(2*z_test, -20.5, .5)
variable_name = 'DEN'
time = 3.5

den = tiegcm.time_interpolate(point, variable_name, time)
den

In [ ]:
from tiegcm.util import scale_height, gravity

T = tiegcm.time_interpolate(point, 'TN', time) # [K]

H_oxygen = scale_height(T, 0, 16)
H_oxygen # [m]

### Boundary height

In [ ]:
z_bndy = tiegcm.time_interpolate(point, 'Z', time) 
point3 = Point3D(1.0001*z_bndy, -20.5, .5)
print point3

### Mixing ratio

In meteorology, mixing ratio usually is defined as the mass of a constituent mi divided by the total mass of all other constituents in a mixture.

$$ mmr = m_i/(m_{tot} - m_i) $$

Therefore, 

$$ m_i = mmr(m_{tot} - m_i) = mmr \cdot m_{tot} - mmr \cdot m_i $$
$$ m_i(1 + mmr) = mmr \cdot m_{tot} $$
$$ m_i = mmr \cdot m_{tot}/(1+mmr) $$

### Mass density of molecular oxygen

In [ ]:
mmr_o = tiegcm.time_interpolate(point, 'O1', time)

den_o = mmr_o*den/(1+mmr_o)
den_o, den

In [ ]:
molecular_mass = dict(O1 = 16, O2 = 32, N2 = 28)

In [ ]:
molecular_mass

In [ ]:
point

# Mass density at any height h

$$ g = g_0 (r_E/(r_E+h))^2 $$

$$ H_X = kT^{bndy}/M_Xg $$

$$ n_X(h) = n_X^{bndy}\times \exp((h^{bndy}-h)/H_X) [m^{-3}] $$

$$ \rho(h) = 1.66053 \times 10^{-27} \times ( 16 \times n_0(h)+ 32 \times n_{O_2}(h) + 28 \times n_{N_2}(h) ) [kg/m^3]$$ 


In [ ]:
density_tot = tiegcm.time_interpolate(point, 'DEN', time) # total mass density
density_tot

In [ ]:
tiegcm.mass_density(point,time)

In [ ]:
molecular_mass

In [ ]:
mass_density(tiegcm, point3, 'O2', time)

In [ ]:
tiegcm.rootgrp.variables['TN'].shape

In [ ]:
tiegcm.ilev

In [ ]:
tiegcm.time_interpolate(point,'N2N', time)

In [ ]:
np.isclose(1e-11, 1e-12)

In [ ]:
tiegcm.rootgrp.variables['time']

# Global time interpolation

TIEGCM files have no date strings in the filename, so interpolation at an arbitrary point in time requires opening the files, extracting and storing their metadata somewhere. In order to handle this, we need a general time interpolator that is responsible for looking at what files are available in a given directory, extracts date-time ranges from them, then opens the appropriate one for interpolation.



In [8]:
ls

Apr2018_progress_report/  ccmc_interpolator.cpp     setup.py
CMakeFiles/               dist/                     somecode.cpp
CMakeLists.txt            example.cpp               temp/
CMakeLists.txt.bak        kameleon.cpp              test_bind.py
LICENSE.txt               kameleon.hpp              tests/
MANIFEST                  lib/                      tiegcm/
README.md                 main.cpp                  tiegcm.sublime-project
TIEGCM.ipynb              nden_cal_at_high_alt.pdf  tiegcm.sublime-workspace
__pycache__/              neutral_mass_density.png  util.pyc
bin/                      pybind11/
build/                    sample_data/


In [14]:
import os
from tiegcm.tiegcm import Model_Manager
from tiegcm.util import time_in_interval

mm = Model_Manager('sample_data/jasoon_shim_052317_IT_10/out')

mm.files
mm.file_times

xlat = -8.81183
xlon = 161.96608
xalt = 361.10342*1e5 #cm
time = 3.5 #ut hours

density = mm.density(xlat, xlon, xalt, pd.Timestamp('2012-10-01 1:00:02'))
print density

4.12224288242e-15


In [11]:
density = mm.density(xlat, xlon, xalt,  pd.Timestamp('2012-10-01 1:00:02'))
density

4.1222428824175376e-15

In [184]:
mm.ut.min()

0.33333333333333331